In [ ]:
import base64
import os
with open("../secrets.json") as secrets:
    secrets_dict = eval(secrets.read())
    open_api_key = base64.b64decode(secrets_dict["openai_api_key"]).decode('ascii')
    os.environ["OPENAI_API_KEY"] = open_api_key
    if "organization_id" in secrets_dict.keys():
        openai_organization = base64.b64decode(secrets_dict["organization_id"]).decode('ascii')
        os.environ["OPENAI_ORGANIZATION"] = openai_organization
del open_api_key, openai_organization, secrets_dict

# Codeas x Tree Sitter

In [ ]:
# Run it just once
# from tree_sitter import Language
# Language.build_library(
#   # Store the library in the `build` directory
#   'build/tree-sitter-grammars.so',

#   # Include one or more languages
#   [
#     '/Users/itortouch/GiTor/tree-sitter-python',
#     '/Users/itortouch/GiTor/tree-sitter-javascript',
#     '/Users/itortouch/GiTor/tree-sitter-java'
#   ]
# )

In [ ]:
# os.chdir("../examples/java_repo_s")
# os.chdir("../examples/ml_repo_s")
os.chdir("../examples/js_repo_s")

In [ ]:
from codeas.assistant import Assistant
codeas = Assistant()

In [ ]:
if os.path.exists(".codeas"):
    os.remove(".codeas/assistant.yaml")
    os.remove(".codeas/prompts.yaml")
    os.rmdir(".codeas")
codeas.init_configs()
codeas.model_post_init(None)

In [ ]:
# TODO: Due to codeas.file_handler._target_files incremental adding and not eliminating values, we can only execute one prompt for each instance session. Need to restart for changing prompt. Review.
# codeas.execute_prompt(instructions="Create java docs.", modules=["StringUtil"])
# codeas.execute_prompt(instructions="Create docstrings.", modules=["ml_pipeline"])
# codeas.execute_prompt(instructions="Create docstrings.", modules=["ClassA"])
# codeas.execute_prompt(instructions="Create unit test.", target="tests", modules=["ClassA"])
codeas.execute_prompt(instructions="Create JS Docs in markdown.", target="docs", modules=["ClassA"])

In [ ]:
codeas.apply_changes()

In [ ]:
codeas.revert_changes()

In [ ]:
codeas.reject_changes()